# Setup of SageMaker Endpoint for DescriptiveWorld computer vision model

Authored by Blair Jones, 2021.11.01

Model built using yolov5

Code inspired by AWS-provided example at: https://aws.amazon.com/blogs/machine-learning/speed-up-yolov4-inference-to-twice-as-fast-on-amazon-sagemaker/



### Instructions

Before running the notebook, install yolov5 in the Sagemaker root folder using the yolov5 instructions at:  https://github.com/ultralytics/yolov5

### Setup

In [55]:
import numpy as np
import time
import json
import requests
import boto3
import os
import sagemaker
from tqdm.notebook import tqdm

In [3]:
from sagemaker import get_execution_role
from sagemaker.session import Session

role = get_execution_role()
sess = Session()
region = sess.boto_region_name
bucket = 's3://descriptiveworld-models/CV_Models/'

In [4]:
print(region)
print(bucket)

us-west-2
s3://descriptiveworld-models/CV_Models/


In [5]:
# ensure that environment chosen for pytorch version >= 1.7 (as of 2021.11.01)

In [7]:
import torch
print(torch.__version__)

1.7.1


In [8]:
import sys
print(sys.version)

3.6.13 | packaged by conda-forge | (default, Feb 19 2021, 05:36:01) 
[GCC 9.3.0]


### Convert model to Torchscript

Run "aws configure" at command line to setup aws CLI.

In [12]:
!aws s3 cp s3://descriptiveworld-models/CV_Models/df2_11_large_20211021/weights/best.pt ../../yolov5/

download: s3://descriptiveworld-models/CV_Models/df2_11_large_20211021/weights/best.pt to ../../yolov5/best.pt


In [22]:
%cd ../../yolov5
!python ./bcj_export.py --weights ./best.pt

[Errno 2] No such file or directory: '../../yolov5'
/home/ec2-user/SageMaker/yolov5
bcj_export: data=data/coco128.yaml, weights=./best.pt, imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=13, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript']
YOLOv5 🚀 v6.0-42-g4c0982a torch 1.7.1 CPU

Fusing layers... 
Model Summary: 367 layers, 46162128 parameters, 0 gradients

PyTorch: starting from best.pt (92.8 MB)

TorchScript: starting export with torch 1.7.1...
/home/ec2-user/SageMaker/yolov5/models/yolo.py:58: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4] or self.onnx_dynamic:
/home/ec2-user/Sag

In [24]:
!tar -czvf ./currentmodel.tar.gz ./current.torchscript.pt

./current.torchscript.pt


#### save model archive to s3

In [25]:
!aws s3 cp ./currentmodel.tar.gz s3://descriptiveworld-models/CV_Models/

upload: ./currentmodel.tar.gz to s3://descriptiveworld-models/CV_Models/currentmodel.tar.gz


### Create the model and endpoint

In [52]:
model_archive = 'currentmodel.tar.gz'
prefix = 's3://descriptiveworld-models/CV_Models/'
model_path = sess.upload_data(path=model_archive, key_prefix=prefix)

In [66]:
model_path

's3://sagemaker-us-west-2-769212126689/s3://descriptiveworld-models/CV_Models//currentmodel.tar.gz'

In [62]:
ls

bcj_export.py        current.torchscript.pt  LICENSE            train.py
bcj_sm_ep_detect.py  data/                   models/            tutorial.ipynb
best.pt              detect.py               README.md          utils/
blank.py             Dockerfile              requirements.txt*  val.py
CONTRIBUTING.md      export.py               runs/
currentmodel.tar.gz  hubconf.py              setup.cfg


In [69]:
%%time
framework_version = '1.7.1'
py_version = 'py3'
instance_type = 'ml.t2.medium'
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

sm_model = PyTorchModel(model_data=model_path,
                       framework_version=framework_version,
                       role=role,
                       sagemaker_session=sess,
                       entry_point='./bcj_sm_ep_detect.py',
                       dependencies=['data', 'models', 'utils'],
                       py_version=py_version,
                       env={"COMPILEDMODEL": 'False', 'MMS_MAX_RESPONSE_SIZE': '100000000', 'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'})
dw_predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type)

-------------!CPU times: user 14.5 s, sys: 1.78 s, total: 16.3 s
Wall time: 6min 49s


In [67]:
print(sm_model.name)
print(dw_predictor.endpoint_name)

pytorch-inference-2021-11-03-20-31-36-313
pytorch-inference-2021-11-03-20-31-36-590


### Run Inference

In [51]:
%%time
iters = 1000
warmup = 100
client = boto3.client('sagemaker-runtime', region_name=region)

content_type = 'application/x-image'

sample_img_url = "https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg"
body = requests.get(sample_img_url).content

dw_perf = []
  
for i in tqdm(range(iters)):
    t0 = time.time()
    response = client.invoke_endpoint(EndpointName=dw_predictor.endpoint_name, Body=body, ContentType=content_type)
    t1 = time.time()
    #convert to millis
    dw_elapsed = (t1-t0)*1000
    
    if warmup == 0:
        dw_perf.append(uncompiled_elapsed)
    else:
        print(f'warmup ({i}, {iters}) : dw - {dw_elapsed} ms')
        warmup = warmup - 1

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 